# Physics 256
## Lecture 07 - Modules, filesystem & I/O

<img src="http://upload.wikimedia.org/wikipedia/commons/d/d4/Pi_pie2.jpg" width=300px>

## Last Time

- functions
- introduction to modules

## Today
- more modules
- working with the filesystem
- input and output


\begin{equation}
\sin(\theta) = \frac{y}{\sqrt{x^2+y^2}}
\end{equation}

### Writing modules

Any code or collection of methods we write can be included in a module by writing them in a text editor (i.e. not in the iPython notebook or interpreter) and saving them with a unqiue filename `mymodule.py`.  We can treat these as complete python programs that can be run, or we can import them from other scripts to maximize effeciency and build collections of useful libraries.

Modules and scripts can both named `mymodule.py` and both can contain variables, functions/methods and classes.

** What's the difference?**

* When imported, a module's `__name__` attribute is set to the module's file name, without *.py*.
* When executed as a script, the `__name__` attribute is set to `__main__`.
* Except for special cases, you shouldn't put any major executable code at the top-level. 
* Put code in functions, classes, methods, and guard it with if `__name__ == "__main__"`.

In [ ]:
# %load data/mymodule.py
'''
A python module to exhibit the use of the __main__ name.
'''

def foo():
    ''' A test function. '''
    return 1

# Main Function
def main():
    # put all your main program driver code here
    print(foo())
    
# main is called once when the script is executed.    
if __name__ == '__main__':
    main()


In [3]:
# then from another script (or from the iPython notebook) 
# we can import the functions defined in this script

# we need to tell python where the script lives, via the sys module
import sys
sys.path.append('data')

import mymodule

mymodule.foo()

1

## Computing $\pi$

Let's put these ideas into practice by writing a module containing functions that compute the value of $\pi$ in various ways.

### Gottfried Wilhelm Leibniz 1646-1716
<img src="http://upload.wikimedia.org/wikipedia/commons/6/6a/Gottfried_Wilhelm_von_Leibniz.jpg" width=200px>

\begin{equation}
\pi = 4 \sum_{n=1}^\infty \frac{(-1)^{n+1}}{2n-1}
\end{equation}

### Abraham Sharp 1653-1742
<img src="http://upload.wikimedia.org/wikipedia/commons/4/43/Sharp_Abraham.jpg" width=200px>
\begin{equation}
\pi = \sum_{n=0}^\infty \frac{2(-1)^{n}3^{1/2-n}}{2n+1}
\end{equation}

### Monte Carlo Calculation using `random.random()`
<img src="http://upload.wikimedia.org/wikipedia/commons/thumb/c/ce/Circle_Area.svg/265px-Circle_Area.svg.png" width="200px">
<img src="http://upload.wikimedia.org/wikipedia/commons/8/84/Pi_30K.gif" width="200px">


In [ ]:
# %load data/pi.py
'''
File: pi.py
Functions that can be used to approximate pi.
'''

def leibniz(N):
    '''Compute pi via an accelerated Leibniz summation.'''
    lpi = 0.0
    for n in range(1,N):
        if n % 2:
            sign = 1
        else:
            sign = -1
        lpi += 1.0*sign/(2*n-1)
    return 4.0*lpi

def sharp(N):
    '''Compute pi via the Sharp summation.'''
    spi = 0.0
    for n in range(N):
        if n % 2:
            sign = -1
        else:
            sign = 1
        num = 2*sign*3**(0.5-n)
        denom = 2*n+1
        spi += num/denom
    return spi

def monte_carlo(N):
    '''Compute pi via Monte Carlo.'''

    import random,math
    inCircle = 0

    for n in range(N):
        x = -0.5 + random.random()
        y = -0.5 + random.random()
        r = math.sqrt(x**2 + y**2)

        if r <= 0.5:
            inCircle += 1

    mcpi = 4.0*inCircle / (1.0*N)
    return mcpi

# The order of our pi approximations
N = 5*10**6

# output to the terminal
print('Leibniz     π = %16.14f' % leibniz(N))
print('Sharp       π = %16.14f' % sharp(N))
print('Monte Carlo π = %16.14f' % monte_carlo(N))



In [5]:
%time %run data/pi.py

Leibniz     π = 3.14159285358982
Sharp       π = 3.14159265358979
Monte Carlo π = 3.14073040000000
CPU times: user 9.18 s, sys: 20.2 ms, total: 9.2 s
Wall time: 9.23 s


In [6]:
%time import pi

Leibniz     π = 3.14159285358982
Sharp       π = 3.14159265358979
Monte Carlo π = 3.14081200000000
CPU times: user 9.08 s, sys: 19.3 ms, total: 9.1 s
Wall time: 9.12 s


In [7]:
import imp
%time imp.reload(pi)

CPU times: user 2.42 ms, sys: 1.98 ms, total: 4.4 ms
Wall time: 3.2 ms


<module 'pi' from 'data/pi.py'>

In [8]:
print(pi.__name__)

pi


Let's compare the accuracy of our various methods

In [9]:
N = [10**n for n in range(3,7)]

# Get the Leibniz, Sharp and Monte Carlo approximations to pi
π = [(pi.leibniz(cN),pi.sharp(cN),pi.monte_carlo(cN)) for cN in N]

# Output the results to the terminal
print('%16s%16s%16s%16s' % ('N','Leibniz  π','Sharp  π','Monte Carlo π'))
for i,cN in enumerate(N):
    print('%16d%16.8f%16.8f%16.8f' % (cN,π[i][0],π[i][1],π[i][2]))

               N      Leibniz  π        Sharp  π   Monte Carlo π
            1000      3.14259365      3.14159265      3.20800000
           10000      3.14169266      3.14159265      3.16000000
          100000      3.14160265      3.14159265      3.13796000
         1000000      3.14159365      3.14159265      3.14435600


## Accessing the Filesytem

It can be very useful to use python to work with the filesystem.  This is most conveniently done with the use of two standard library modules:

In [2]:
import os
import shutil

### List all files in a directory

In [12]:
list_path = '/Users/agdelma/Documents/Phys256/Lectures/Notebooks/data'
for f in os.listdir(list_path):
    print(f)

__pycache__
function.png
in.dat
leapyear.py
lists.png
mymodule.py
out.dat
pi.py
primes.py
sho_energy.dat


In [14]:
# what if we only want the files (and not directories?)
for f in os.listdir(list_path):
    if os.path.isfile(os.path.join(list_path,f)):
        print(f)

function.png
in.dat
leapyear.py
lists.png
mymodule.py
out.dat
pi.py
primes.py
sho_energy.dat


### Creating a directory

In [15]:
example_num = 2
dir_name = 'examples' + os.path.sep + 'example%02d' % example_num 
if not os.path.exists(dir_name):
    os.makedirs(dir_name)

### Copy (or move) files

In [16]:
for f in os.listdir(list_path):
    extension = os.path.splitext(f)[-1]
    if extension == '.py':
        dest = dir_name + os.path.sep + f
        src = list_path + os.path.sep + f
        shutil.copy(src,dest)  

### Can even create archive files (see the `shutil` help docs)

In [17]:
shutil.make_archive('example_archive', 'zip', '.', 'examples')

'/Users/agdelma/Documents/Phys256/Lectures/Notebooks/example_archive.zip'

<div class="span alert alert-success">
<h2> Team programming challenge </h2> <br />

Write a python script that produces the appropriately named .zip archive for your assignment
solutions.

You will need to:
<ul>
<li>specify a source directory where your solutions are located</li>
<li>specify the assignment number</li>
<li>create the target directories: Lastname_I/A0X/</li>
<li>copy the solution files into the target directory</li>
<li>create the .zip file</li>
<li>perform any required clean up </li>
</ul>
</div>

In [ ]:
# %load data/makezip.py
''' Make a zipped directory for Physics 256 Assignments.
'''

import os, shutil


def main():

    assign_num = 2
    solution_path = 'files'
    name = 'DelMaestro_A'

    # get the list of all files
    assign_files  = [f for f in os.listdir(solution_path) if os.path.isfile(os.path.join(solution_path,f)) ]

    # test if the assignment directory exists, if not, make it
    assign_dir = name + os.path.sep + 'A%02d' % assign_num
    if not os.path.exists(assign_dir):
        os.makedirs(assign_dir)

    # copy them to the assignment directory
    for afile in assign_files:
        source = solution_path + os.path.sep + afile
        dest = assign_dir + os.path.sep + afile
        shutil.copy(source,dest)

    # make the archive
    shutil.make_archive(name + '_A%02d' % assign_num, 'zip', '.', name)

if __name__ == '__main__':
    main()


### Expanding home directory shortcut


In [6]:
print(os.path.expanduser('~/Documents'))

/Users/agdelma/Documents


## Input & Output

So far, we have mostly been setting the values of variables used in our programs by hand and writing things to the terminal with the `print()` function.  We can use python to easily get data from a user, either form the command line, or from a file, and save our program output to a well-formatted file.

In [7]:
# asking the user for input
N = input("Number of elements: ")
print(N)

Number of elements: 10
10


In [8]:
# all input is treating like a string
print(type(N))

<class 'str'>


In [9]:
# we must manually convert if we need numerical data
Ni = int(N)
print(Ni)

Nf = float(N)
print(Nf)

N = float(input("enter a float: "))
print(type(N))

10
10.0
enter a float: 123.124
<class 'float'>


### Working with files
We can use the keyword `open` to open a file, but we must specify the mode, either `r` for read, `w` for write `r+` for read/write and `a` for append

In [10]:
!cat 'data/in.dat'

1
2
3
4
5
6
7
8
9
10


In [11]:
in_file = open('data/in.dat', 'r')

In [12]:
# reading an entire file
print(in_file.read())

# we close when we are finished
in_file.close()

1
2
3
4
5
6
7
8
9
10



In [13]:
# alternatively, we can open a file within a context
with open('data/in.dat', 'r') as f:
    read_data = f.read()
    print(read_data)
    
# the file is automatically closed when we leave the with context
print(f.closed)

1
2
3
4
5
6
7
8
9
10

True


In [14]:
# we can get a list of all lines
with open('data/in.dat', 'r') as in_file:
    lines = in_file.readlines()

print(lines)

['1\n', '2\n', '3\n', '4\n', '5\n', '6\n', '7\n', '8\n', '9\n', '10\n']


<div class="span alert alert-danger">
Note that the newline characters are included!
</div>

In [15]:
# or just a single line
with open('data/in.dat', 'r') as in_file:
    line = in_file.readline()
print(line)

1



In [19]:
# let's iterate through all the lines in the file
in_file = open('data/in.dat', 'r')
lines = in_file.readlines()
in_file.close()
for line in lines:
    print (line.rstrip('\n'), end=' ')

1 2 3 4 5 6 7 8 9 10 

### Writing to files

In [20]:
# specifiying 'w' will create a file if it doesn't exist and replace the content if it does
out_file = open('data/out.dat', 'w')

In [21]:
# we can only write strings to files, so we must convert on input and output
for line in lines:
    a = int(line)
    b = a**2
    out_file.write('%d\n' % b)
out_file.close()   

In [22]:
!cat data/out.dat

1
4
9
16
25
36
49
64
81
100


### Output formatting

Python supports various ways of formatting i/o. We have already seen the c-style `%` method which is my preferred method. There is a new-fangled way which is more flexible and
powerful,see: http://docs.python.org/tutorial/inputoutput.html use whichever you like for assignments.

A fully descriptive list of all formatting options can be found at https://docs.python.org/3/library/stdtypes.html#string-formatting

<div class="span alert alert-success">
<h2> Team programming challenge </h2>
<h3> Average energy for the simple harmonic oscillator at finite temperature</h3>
</div>

I have uploaded a data file on BlackBoard  called `sho_energy.dat`.  The line contains column headings with units in kelvin.

The next set of lines contain quantum Monte Carlo measurements for the kinetic and potential energy of the simple harmonic osscilator at $T = 0.5~\mathrm{K}$ where $\hbar \omega/k_{\mathrm{B}} = 1$.  The exact answer is known to be:
\begin{equation}
E(T) = \frac{\hbar \omega}{2} \coth \frac{\hbar \omega}{2 k_{\mathrm{B}} T}.
\end{equation}

Download the data file to your computer and write a program that loads the file from disk and stores it in a dictionary with labels taken from the column headers.  Compute the average total energy of all lines.  If you have extra time, compare with the exact result.

In [23]:
!cat data/sho_energy.dat

#         Kinetic       Potential 
   6.22407909E-01  3.77134851E-01  
   4.29547101E-01  2.42379252E-01  
   2.37001671E-01  2.67495403E-01  
   3.01818649E-01  2.58392255E-01  
   4.09565316E-01  3.15222495E-01  
   2.14317707E-01  4.51218376E-01  
   4.18641845E-01  2.89516631E-01  
   3.66230261E-01  2.80669596E-01  
   2.39985912E-01  3.49197015E-01  
   4.08868551E-01  2.73554341E-01  
   4.29891894E-01  2.28270437E-01  
   1.76880098E-01  3.74807008E-01  
   4.50146777E-01  2.60430892E-01  
   2.94294733E-01  3.82058897E-01  
   4.52161080E-01  2.44881829E-01  
   3.44609451E-01  3.31618376E-01  
   4.01977123E-01  2.86437236E-01  
   1.95875270E-01  3.09855155E-01  
   4.07812408E-01  3.67230198E-01  
   2.81732855E-01  3.38656673E-01  
   3.49238393E-01  2.34915354E-01  
   7.58970450E-02  3.47864555E-01  
   2.35324623E-01  2.41883096E-01  
   4.14538771E-01  4.14814837E-01  
   3.56186180E-01  2.82852947E-01  
   3.08263779E-01  2.61148485E-01  
   3.51531602E-01  3.84750836